In [1]:
import pandas as pd
from spacy.cli import download
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import spacy

In [2]:
train_data = pd.read_csv('tmp/train_data.csv')

In [3]:
train_data.head()

,comments_url,id,title,body,issue_url,pr_url,labels,pr_number,filename,status,additions,deletions,changes,all_comments,all_text
0,https://api.github.com/repos/kubernetes/kubern...,2639668210,kubelet crash: fatal error: concurrent map writes,### What happened?\n\nWhile looking into three...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],128657,"['pkg/kubelet/cm/container_manager_linux.go', ...","['modified', 'modified', 'modified', 'modified']","[3, 4, 18, 75]","[3, 3, 6, 0]","[6, 7, 24, 75]",/sig node Thought to search for similar errors...,kubelet fatal concurrent map writes while look...
1,https://api.github.com/repos/kubernetes/kubern...,2617512099,[FG:InPlacePodVerticalScaling] failed to verif...,### What happened?\n\nOne line bug description...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],126620,"['pkg/kubelet/status/state/checkpoint.go', 'pk...","['modified', 'modified', 'added']","[42, 23, 166]","[26, 17, 0]","[68, 40, 166]",This issue is currently awaiting triage.\n\nIf...,failed verify pod status checkpoint checksum b...
2,https://api.github.com/repos/kubernetes/kubern...,2604613192,Restore build-tag flag for code-generator,### What happened?\n\nThe `build-tag` flag is ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/code-generation'],128259,['staging/src/k8s.io/code-generator/cmd/conver...,"['modified', 'modified', 'modified', 'modified...","[12, 1, 1, 12, 1, 1]","[2, 1, 1, 1, 1, 1]","[14, 2, 2, 13, 2, 2]",@p0lyn0mial FYI /sig api-machinery /triage acc...,restore flag flag removed moved removed depend...
3,https://api.github.com/repos/kubernetes/kubern...,2596132738,[Failing Tests] ci-crio-cgroupv1-node-e2e-conf...,### Which jobs are failing?\n* master-blocking...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/kubeadm', 'area/dependency']",128175,"['go.mod', 'go.sum', 'hack/unwanted-dependenci...","['modified', 'modified', 'modified', 'modified...","[1, 3, 1, 21, 9, 9, 4, 0, 10, 2, 2, 5, 8, 1]","[1, 2, 0, 98, 8, 7, 6, 62, 11, 1, 4, 8, 11, 1]","[2, 5, 1, 119, 17, 16, 10, 62, 21, 3, 6, 13, 1...",@drewhagen: The provided milestone is not vali...,multiple which jobs are which tests are node c...
4,https://api.github.com/repos/kubernetes/kubern...,2591358936,Crash on kube manager's service-lb-controller ...,### What happened?\n\nIf kube manager is start...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/cloudprovider'],128182,['cmd/kube-controller-manager/app/controllerma...,"['modified', 'modified', 'modified']","[26, 12, 5]","[0, 1, 4]","[26, 13, 9]",This issue is currently awaiting triage.\n\nIf...,crash kube after kube manager started no looks...


In [4]:
for i in range(5) : 
    print(train_data['all_text'][i])

kubelet fatal concurrent map writes while looking into three failing e2e tests allalpha found that kubelet had crashed nov 07 fatal concurrent map writes nov 07 goroutine 29497 nov 07 nov 07 nov 07 nov 07 nov 07 nov 07 nov 07 nov 07 nov 07 nov 07 nov 07 nov 07 nov 07 nov 07 nov 07 nov 07 nov 07 nov 07 nov 07 nov 07 nov 07 created by goroutine 250 nov 07 did expect no can but have logs from prow job that encountered that crash requires particular alpha feature since was an allalpha _no response_ gce paste output here paste output here wmic get osarchitecture paste output here container runtime plugins versions
failed verify pod status checkpoint checksum because different behaviors func one line content func writes into file different from func reads from same which causes function always fails some format quantity did expect kubelet crash can start normally can prepare pod spec content below 80 copy pod spec into dir master node restart kubelet systemctl restart kubelet kubelet will cr

Check the label distribution

In [5]:
# Flatten the list of labels if it's a list of lists
flattened_labels = []
for label_list in train_data["labels"]:
    # Convert string representation of list to actual list
    if isinstance(label_list, str):
        # Remove brackets and split by comma
        clean_label_list = label_list.strip('[]').replace("'", "").split(', ')
    else:
        clean_label_list = label_list
    
    # Add each label to the flattened list
    flattened_labels.extend(clean_label_list)

# Print labels distribution count
label_counts = Counter(flattened_labels)
print("Label Distribution Count (Training Data):")

for i, (label, count) in enumerate(label_counts.most_common()):
    print(f"{i+1}. {label}: {count}")

Label Distribution Count (Training Data):
1. area/test: 309
2. area/kubelet: 304
3. area/apiserver: 196
4. area/cloudprovider: 171
5. area/kubectl: 133
6. area/dependency: 80
7. area/code-generation: 65
8. area/provider/azure: 64
9. area/ipvs: 39
10. area/kubeadm: 33
11. area/provider/gcp: 32
12. area/e2e-test-framework: 28
13. area/kube-proxy: 27
14. area/release-eng: 23
15. area/conformance: 12
16. area/custom-resources: 6
17. area/security: 5
18. area/etcd: 5
19. area/deflake: 3
20. area/network-policy: 3
21. area/code-organization: 2
22. area/api: 1
23. area/batch: 1
24. area/client-libraries: 1
25. area/provider/aws: 1


Stopwords Removal

In [6]:
# remove stopwords using spaCy
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    download('en_core_web_sm')
    nlp = spacy.load('en_core_web_sm')

train_data['all_text'] = train_data['all_text'].apply(lambda x: ' '.join([word.text for word in nlp(x) if not word.is_stop]))

In [7]:
# check null values in all_text column
# Display null values for all columns
print(f"Null all_text column : {train_data['all_text'].isnull().sum()}")

Null all_text column : 0


Data Lemmatization

In [8]:
# lematize the text using spaCy
train_data['all_text'] = train_data['all_text'].apply(lambda x: ' '.join([word.lemma_ for word in nlp(x)]))

In [9]:
# check all_text column for null values
print(f"Null all_text column : {train_data['all_text'].isnull().sum()}")

Null all_text column : 0


Add filename to the text

In [10]:
# add filename to the data
# Only append filename if it exists (not null/None)
train_data['all_text'] = train_data.apply(lambda row: row['all_text'] + ' ' + row['filename'] if pd.notna(row['filename']) else row['all_text'], axis=1)

In [11]:
# check all_text column for null values
print(f"Null all_text column : {train_data['all_text'].isnull().sum()}")

Null all_text column : 0


# Export the Data

In [12]:
train_data.head()

,comments_url,id,title,body,issue_url,pr_url,labels,pr_number,filename,status,additions,deletions,changes,all_comments,all_text
0,https://api.github.com/repos/kubernetes/kubern...,2639668210,kubelet crash: fatal error: concurrent map writes,### What happened?\n\nWhile looking into three...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],128657,"['pkg/kubelet/cm/container_manager_linux.go', ...","['modified', 'modified', 'modified', 'modified']","[3, 4, 18, 75]","[3, 3, 6, 0]","[6, 7, 24, 75]",/sig node Thought to search for similar errors...,kubelet fatal concurrent map write look fail e...
1,https://api.github.com/repos/kubernetes/kubern...,2617512099,[FG:InPlacePodVerticalScaling] failed to verif...,### What happened?\n\nOne line bug description...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],126620,"['pkg/kubelet/status/state/checkpoint.go', 'pk...","['modified', 'modified', 'added']","[42, 23, 166]","[26, 17, 0]","[68, 40, 166]",This issue is currently awaiting triage.\n\nIf...,fail verify pod status checkpoint checksum dif...
2,https://api.github.com/repos/kubernetes/kubern...,2604613192,Restore build-tag flag for code-generator,### What happened?\n\nThe `build-tag` flag is ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/code-generation'],128259,['staging/src/k8s.io/code-generator/cmd/conver...,"['modified', 'modified', 'modified', 'modified...","[12, 1, 1, 12, 1, 1]","[2, 1, 1, 1, 1, 1]","[14, 2, 2, 13, 2, 2]",@p0lyn0mial FYI /sig api-machinery /triage acc...,restore flag flag remove move remove dependenc...
3,https://api.github.com/repos/kubernetes/kubern...,2596132738,[Failing Tests] ci-crio-cgroupv1-node-e2e-conf...,### Which jobs are failing?\n* master-blocking...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/kubeadm', 'area/dependency']",128175,"['go.mod', 'go.sum', 'hack/unwanted-dependenci...","['modified', 'modified', 'modified', 'modified...","[1, 3, 1, 21, 9, 9, 4, 0, 10, 2, 2, 5, 8, 1]","[1, 2, 0, 98, 8, 7, 6, 62, 11, 1, 4, 8, 11, 1]","[2, 5, 1, 119, 17, 16, 10, 62, 21, 3, 6, 13, 1...",@drewhagen: The provided milestone is not vali...,multiple job test node conformance test testgr...
4,https://api.github.com/repos/kubernetes/kubern...,2591358936,Crash on kube manager's service-lb-controller ...,### What happened?\n\nIf kube manager is start...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/cloudprovider'],128182,['cmd/kube-controller-manager/app/controllerma...,"['modified', 'modified', 'modified']","[26, 12, 5]","[0, 1, 4]","[26, 13, 9]",This issue is currently awaiting triage.\n\nIf...,crash kube kube manager start look like instan...


In [13]:
# count nan values in the train_data
train_data.isna().sum()

comments_url     0
id               0
title            0
body             0
issue_url        0
pr_url           0
labels           0
pr_number        0
filename         0
status           0
additions        0
deletions        0
changes          0
all_comments    27
all_text         0
dtype: int64

In [14]:
# # export the train_data and val_data to csv
train_data.to_csv('tmp/preprocessed data/preprocessed_train_data.csv', index=False)
